In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import random
import time

# Initialize mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

# Eye positions on canvas
left_eye_pos = (220, 200)
right_eye_pos = (420, 200)
eye_radius = 60
pupil_radius = 20
blink_timer = time.time()
is_blinking = False

# Blinking logic
def maybe_blink():
    global is_blinking, blink_timer
    if time.time() - blink_timer > random.uniform(3, 6):  # Blink every 3–6 sec
        is_blinking = True
        blink_timer = time.time()
    elif is_blinking and time.time() - blink_timer > 0.15:
        is_blinking = False

# Draw one stylized eye
def draw_stylized_eye(canvas, center, iris_offset=(0, 0), blink=False):
    if blink:
        # Simulate eyelid by drawing a filled ellipse
        cv2.ellipse(canvas, center, (eye_radius, 10), 0, 0, 360, (180, 180, 180), -1)
        return

    # Draw eye white with border
    cv2.circle(canvas, center, eye_radius, (220, 220, 220), -1)  # light gray eye white
    cv2.circle(canvas, center, eye_radius, (50, 50, 50), 2)      # black border

    # Compute pupil position
    pupil_center = (center[0] + iris_offset[0], center[1] + iris_offset[1])

    # Draw pupil with fake shading
    cv2.circle(canvas, pupil_center, pupil_radius, (0, 0, 0), -1)
    highlight_center = (pupil_center[0] - 5, pupil_center[1] - 5)
    cv2.circle(canvas, highlight_center, 5, (200, 200, 200), -1)  # eye shine

    # Optional: draw iris circle
    iris_radius = pupil_radius + 5
    cv2.circle(canvas, pupil_center, iris_radius, (90, 90, 180), 2)  # iris outline

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # mirror image
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    canvas = np.ones((400, 640, 3), dtype=np.uint8) * 255  # white background

    maybe_blink()

    iris_offset_left = (0, 0)
    iris_offset_right = (0, 0)

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        def get_px(idx):
            lm = landmarks[idx]
            return int(lm.x * w), int(lm.y * h)

        # Use tip of nose (index 1) as the face center reference
        face_center = get_px(1)

        # Increase movement multiplier and range here
        def calc_offset(eye_pos):
            dx = face_center[0] - w // 2
            dy = face_center[1] - h // 2
            dx = int(np.clip(dx * 0.20, -25, 25))  # 👁️ More movement
            dy = int(np.clip(dy * 0.20, -25, 25))
            return dx, dy

        iris_offset_left = calc_offset(left_eye_pos)
        iris_offset_right = calc_offset(right_eye_pos)

    # Draw stylized cartoon eyes
    draw_stylized_eye(canvas, left_eye_pos, iris_offset_left, is_blinking)
    draw_stylized_eye(canvas, right_eye_pos, iris_offset_right, is_blinking)

    cv2.imshow("Big-Movement Cartoon Eyes", canvas)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 